In [1]:
import pandas as pd
import numpy as np
from stockstats import StockDataFrame as Sdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from OHCLV_loader import load_data
from datetime import datetime
from pytz import timezone
from financial_loader import getFundamental, getFinancialReports, getFinancialReport, getBasicFA
from valuation_loader import get_pe_valuation, get_pe_valuations, get_pb_valuation, get_pb_valuations, estimate_price
from utils.date_util import *
from analysis.CANSLIM_analysis import CANSLIM_analysis
import requests

# from preprocess.dataloader import load_data, clean_data
# from preprocess.data_generator import generate_trading_data
# from preprocess.indicator_factory import add_indicators_all, add_indicators
# from indicators.Trendline import FindTrends
# from evals import eval_trend_return_acc



In [2]:
data_src = "data/Vietnam"
stock_list = np.load(data_src + "/list_stocks.npy", allow_pickle = True)
stock_infos = pd.read_csv("data/Vietnam/stock_infos.csv", index_col = "ticker")
basicFA = pd.read_csv("data/Vietnam/basic_FA.csv", index_col = "ticker")
trailing_quarter = pd.read_csv("data/Vietnam/trailing_quarter_report.csv", index_col = "ticker")
trailing_year = pd.read_csv("data/Vietnam/trailing_year_report.csv", index_col = "ticker")
quarters, years = get_last_quarters("Q2/2023", num_periods = 26)

estimate_price_pe = pd.read_csv("data/Vietnam/estimate_price_pe.csv", index_col = "ticker")
estimate_price_pb = pd.read_csv("data/Vietnam/estimate_price_pb.csv", index_col = "ticker")

price_estimator = estimate_price(stock_list, stock_infos, estimate_price_pe, estimate_price_pb)

stock_list = list(trailing_quarter.index.unique())

# stock_list = [e for e in stock_list if e != "L14"]




In [3]:
start_date = get_date_by_report_quarter(quarters[0])[0].strftime("%Y-%m-%d")
end_date = get_date_by_report_quarter(quarters[-1])[1].strftime("%Y-%m-%d")

In [4]:
getFinancialReport("VCB", start_quarter = 2, start_year = 2014, end_quarter = 4, end_year = 2022, mode = "quarter")

<Response [200]>


KeyError: 'quarter'

In [ ]:
print(price_estimator)

In [ ]:
df = get_pe_valuations(["VCB", "VIC"], quarters)

In [ ]:
df

In [ ]:
estimate_price_pe

In [ ]:
estimate_price_pe.loc["VCB"]["Q4/2016"]

In [ ]:
quarters = estimate_price_pe.columns[1:]
results = []
for stock in stock_list:
    res = [stock]
    estimate_price_pes = estimate_price_pe.loc[stock]
    estimate_price_pbs = estimate_price_pb.loc[stock]
    stock_category = stock_infos.loc[stock]["category"]
    
    for quarter in quarters:
        estimate_PE_range = estimate_price_pes[quarter]
        estimate_PB_range = estimate_price_pbs[quarter]
        
        estimate_PE_low = float(estimate_PE_range.split(',')[0])
        estimate_PE_high = float(estimate_PE_range.split(',')[1])
        
        estimate_PB_low = float(estimate_PB_range.split(',')[0])
        estimate_PB_high = float(estimate_PB_range.split(',')[1])
        
        if stock_category in ["Banks", "Financial_Services", "Insurance"]:    
            pb_weight = 0.62
            pe_weight = 0.38
        else:
            pb_weight = 0.38
            pe_weight = 0.62

        if estimate_PE_low > 0:
            estimated_price_low = estimate_PE_low * pe_weight + estimate_PB_low * pb_weight
        else:
            estimated_price_low = estimate_PB_low

        if estimate_PE_high > 0:
            estimated_price_high = estimate_PE_high * pe_weight + estimate_PB_high * pb_weight
        else:
            estimated_price_high = estimate_PB_high

        res.append("{:.0f}, {:.0f}".format(estimated_price_low, estimated_price_high))
    results.append(res)
    
df = pd.DataFrame(results, columns = ["ticker"] + list(quarters))

print(df)
    
    
    
